In [2]:
import pandas as pd
import numpy as np
import skfuzzy as fuzz
import skfuzzy.control as ctrl
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Define fuzzy logic system 1
def create_fuzzy_system_1():
    income = ctrl.Antecedent(np.arange(0, 100001, 1000), 'income')
    debt_to_income = ctrl.Antecedent(np.arange(0, 101, 1), 'debt_to_income')
    credit_score = ctrl.Antecedent(np.arange(300, 851, 1), 'credit_score')
    risk = ctrl.Consequent(np.arange(0, 11, 1), 'risk')
    
    # Membership functions for fuzzy system 1
    income['low'] = fuzz.trimf(income.universe, [0, 0, 50000])
    income['high'] = fuzz.trimf(income.universe, [30000, 100000, 100000])
    debt_to_income['low'] = fuzz.trimf(debt_to_income.universe, [0, 0, 40])
    debt_to_income['high'] = fuzz.trimf(debt_to_income.universe, [20, 100, 100])
    credit_score['low'] = fuzz.trimf(credit_score.universe, [300, 300, 600])
    credit_score['high'] = fuzz.trimf(credit_score.universe, [550, 850, 850])
    risk['low'] = fuzz.trimf(risk.universe, [0, 0, 5])
    risk['high'] = fuzz.trimf(risk.universe, [5, 10, 10])
    
    # Define rules for fuzzy system 1
    rule1 = ctrl.Rule(income['high'] & debt_to_income['low'] & credit_score['high'], risk['low'])
    rule2 = ctrl.Rule(income['low'] & debt_to_income['high'] & credit_score['low'], risk['high'])
    rule3 = ctrl.Rule(income['high'] & debt_to_income['high'] & credit_score['high'], risk['low'])
    rule4 = ctrl.Rule(income['low'] & debt_to_income['low'] & credit_score['high'], risk['low'])
    rule5 = ctrl.Rule(income['low'] & debt_to_income['high'] & credit_score['high'], risk['high'])
    
    risk_ctrl = ctrl.ControlSystem([rule1, rule2, rule3, rule4, rule5])
    return ctrl.ControlSystemSimulation(risk_ctrl)

# Define fuzzy logic system 2
def create_fuzzy_system_2():
    income = ctrl.Antecedent(np.arange(0, 100001, 1000), 'income')
    debt_to_income = ctrl.Antecedent(np.arange(0, 101, 1), 'debt_to_income')
    credit_score = ctrl.Antecedent(np.arange(300, 851, 1), 'credit_score')
    risk = ctrl.Consequent(np.arange(0, 11, 1), 'risk')
    
    # Membership functions for fuzzy system 2
    income['low'] = fuzz.trimf(income.universe, [0, 0, 50000])
    income['middle'] = fuzz.trimf(income.universe, [20000, 50000, 80000])
    income['high'] = fuzz.trimf(income.universe, [30000, 100000, 100000])
    debt_to_income['low'] = fuzz.trimf(debt_to_income.universe, [0, 0, 40])
    debt_to_income['middle'] = fuzz.trimf(debt_to_income.universe, [20, 50, 80])
    debt_to_income['high'] = fuzz.trimf(debt_to_income.universe, [20, 100, 100])
    credit_score['low'] = fuzz.trimf(credit_score.universe, [300, 300, 600])
    credit_score['middle'] = fuzz.trimf(credit_score.universe, [500, 675, 800])
    credit_score['high'] = fuzz.trimf(credit_score.universe, [550, 850, 850])
    risk['low'] = fuzz.trimf(risk.universe, [0, 0, 5])
    risk['middle'] = fuzz.trimf(risk.universe, [3, 5, 7])
    risk['high'] = fuzz.trimf(risk.universe, [5, 10, 10])
    
    # Define rules for fuzzy system 2
    rule1 = ctrl.Rule(income['high'] & debt_to_income['low'] & credit_score['high'], risk['low'])
    rule2 = ctrl.Rule(income['low'] & debt_to_income['high'] & credit_score['low'], risk['high'])
    rule3 = ctrl.Rule(income['middle'] & debt_to_income['middle'] & credit_score['middle'], risk['middle'])
    rule4 = ctrl.Rule(income['high'] & debt_to_income['high'] & credit_score['high'], risk['low'])
    rule5 = ctrl.Rule(income['low'] & debt_to_income['low'] & credit_score['high'], risk['low'])
    rule6 = ctrl.Rule(income['middle'] & debt_to_income['high'] & credit_score['low'], risk['high'])
    rule7 = ctrl.Rule(income['low'] & debt_to_income['middle'] & credit_score['middle'], risk['middle'])
    rule8 = ctrl.Rule(income['high'] & debt_to_income['middle'] & credit_score['middle'], risk['low'])
    
    risk_ctrl = ctrl.ControlSystem([rule1, rule2, rule3, rule4, rule5, rule6, rule7, rule8])
    return ctrl.ControlSystemSimulation(risk_ctrl)

# Function to generate synthetic data based on a fuzzy control system
def generate_synthetic_data(fuzzy_system, n_samples=100):
    np.random.seed(42)  # For reproducibility
    samples = []
    for _ in range(n_samples):
        income_val = np.random.randint(0, 100001)
        dti_val = np.random.randint(0, 101)
        credit_score_val = np.random.randint(300, 851)
        
        # Set inputs to the fuzzy system
        fuzzy_system.input['income'] = income_val
        fuzzy_system.input['debt_to_income'] = dti_val
        fuzzy_system.input['credit_score'] = credit_score_val
        
        try:
            fuzzy_system.compute()
            risk_val = fuzzy_system.output['risk']
            samples.append([income_val, dti_val, credit_score_val, risk_val])
        except KeyError as e:
            print(f"Skipping sample due to error: {e}")
    
    return pd.DataFrame(samples, columns=['income', 'debt_to_income', 'credit_score', 'risk'])

# Generate synthetic data for both systems
fuzzy_system_1 = create_fuzzy_system_1()
fuzzy_system_2 = create_fuzzy_system_2()

data1 = generate_synthetic_data(fuzzy_system_1, n_samples=100)
data2 = generate_synthetic_data(fuzzy_system_2, n_samples=100)

# Display a few rows of the data generated by each system
print("\n### Synthetic Data from Fuzzy Logic System 1 ###")
print(data1.head())

print("\n### Synthetic Data from Fuzzy Logic System 2 ###")
print(data2.head())

# Train and evaluate models on data from each fuzzy system
def train_and_evaluate(data):
    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(
        data[['income', 'debt_to_income', 'credit_score']], data['risk'], test_size=0.3, random_state=42)

    # Train a Random Forest model
    rf = RandomForestRegressor(n_estimators=100, random_state=42)
    rf.fit(X_train, y_train)

    # Predict and evaluate
    y_pred = rf.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    return mse, r2

# Evaluate both systems
mse1, r2_1 = train_and_evaluate(data1)
mse2, r2_2 = train_and_evaluate(data2)

print("\n### Fuzzy Logic System 1 Evaluation ###")
print(f"Mean Squared Error (MSE): {mse1:.2f}")
print(f"R-squared (R2): {r2_1:.2f}")

print("\n### Fuzzy Logic System 2 Evaluation ###")
print(f"Mean Squared Error (MSE): {mse2:.2f}")
print(f"R-squared (R2): {r2_2:.2f}")


Skipping sample due to error: 'risk'
Skipping sample due to error: 'risk'
Skipping sample due to error: 'risk'
Skipping sample due to error: 'risk'
Skipping sample due to error: 'risk'
Skipping sample due to error: 'risk'
Skipping sample due to error: 'risk'
Skipping sample due to error: 'risk'
Skipping sample due to error: 'risk'
Skipping sample due to error: 'risk'
Skipping sample due to error: 'risk'
Skipping sample due to error: 'risk'
Skipping sample due to error: 'risk'
Skipping sample due to error: 'risk'
Skipping sample due to error: 'risk'
Skipping sample due to error: 'risk'
Skipping sample due to error: 'risk'
Skipping sample due to error: 'risk'
Skipping sample due to error: 'risk'
Skipping sample due to error: 'risk'
Skipping sample due to error: 'risk'
Skipping sample due to error: 'risk'
Skipping sample due to error: 'risk'
Skipping sample due to error: 'risk'
Skipping sample due to error: 'risk'
Skipping sample due to error: 'risk'
Skipping sample due to error: 'risk'
S

In [4]:
import pandas as pd
import numpy as np
import skfuzzy as fuzz
import skfuzzy.control as ctrl
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Define fuzzy logic system 1
def create_fuzzy_system_1():
    income = ctrl.Antecedent(np.arange(0, 100001, 1000), 'income')
    debt_to_income = ctrl.Antecedent(np.arange(0, 101, 1), 'debt_to_income')
    credit_score = ctrl.Antecedent(np.arange(300, 851, 1), 'credit_score')
    risk = ctrl.Consequent(np.arange(0, 11, 1), 'risk')
    
    # Membership functions for fuzzy system 1
    income['low'] = fuzz.trimf(income.universe, [0, 0, 50000])
    income['high'] = fuzz.trimf(income.universe, [30000, 100000, 100000])
    debt_to_income['low'] = fuzz.trimf(debt_to_income.universe, [0, 0, 40])
    debt_to_income['high'] = fuzz.trimf(debt_to_income.universe, [20, 100, 100])
    credit_score['low'] = fuzz.trimf(credit_score.universe, [300, 300, 600])
    credit_score['high'] = fuzz.trimf(credit_score.universe, [550, 850, 850])
    risk['low'] = fuzz.trimf(risk.universe, [0, 0, 5])
    risk['high'] = fuzz.trimf(risk.universe, [5, 10, 10])
    
    # Define rules for fuzzy system 1
    rule1 = ctrl.Rule(income['high'] & debt_to_income['low'] & credit_score['high'], risk['low'])
    rule2 = ctrl.Rule(income['low'] & debt_to_income['high'] & credit_score['low'], risk['high'])
    rule3 = ctrl.Rule(income['high'] & debt_to_income['high'] & credit_score['high'], risk['low'])
    rule4 = ctrl.Rule(income['low'] & debt_to_income['low'] & credit_score['high'], risk['low'])
    rule5 = ctrl.Rule(income['low'] & debt_to_income['high'] & credit_score['high'], risk['high'])
    
    risk_ctrl = ctrl.ControlSystem([rule1, rule2, rule3, rule4, rule5])
    return ctrl.ControlSystemSimulation(risk_ctrl)

# Define fuzzy logic system 2
def create_fuzzy_system_2():
    income = ctrl.Antecedent(np.arange(0, 100001, 1000), 'income')
    debt_to_income = ctrl.Antecedent(np.arange(0, 101, 1), 'debt_to_income')
    credit_score = ctrl.Antecedent(np.arange(300, 851, 1), 'credit_score')
    risk = ctrl.Consequent(np.arange(0, 11, 1), 'risk')
    
    # Membership functions for fuzzy system 2
    income['low'] = fuzz.trimf(income.universe, [0, 0, 50000])
    income['middle'] = fuzz.trimf(income.universe, [20000, 50000, 80000])
    income['high'] = fuzz.trimf(income.universe, [30000, 100000, 100000])
    debt_to_income['low'] = fuzz.trimf(debt_to_income.universe, [0, 0, 40])
    debt_to_income['middle'] = fuzz.trimf(debt_to_income.universe, [20, 50, 80])
    debt_to_income['high'] = fuzz.trimf(debt_to_income.universe, [20, 100, 100])
    credit_score['low'] = fuzz.trimf(credit_score.universe, [300, 300, 600])
    credit_score['middle'] = fuzz.trimf(credit_score.universe, [500, 675, 800])
    credit_score['high'] = fuzz.trimf(credit_score.universe, [550, 850, 850])
    risk['low'] = fuzz.trimf(risk.universe, [0, 0, 5])
    risk['middle'] = fuzz.trimf(risk.universe, [3, 5, 7])
    risk['high'] = fuzz.trimf(risk.universe, [5, 10, 10])
    
    # Define rules for fuzzy system 2
    rule1 = ctrl.Rule(income['high'] & debt_to_income['low'] & credit_score['high'], risk['low'])
    rule2 = ctrl.Rule(income['low'] & debt_to_income['high'] & credit_score['low'], risk['high'])
    rule3 = ctrl.Rule(income['middle'] & debt_to_income['middle'] & credit_score['middle'], risk['middle'])
    rule4 = ctrl.Rule(income['high'] & debt_to_income['high'] & credit_score['high'], risk['low'])
    rule5 = ctrl.Rule(income['low'] & debt_to_income['low'] & credit_score['high'], risk['low'])
    rule6 = ctrl.Rule(income['middle'] & debt_to_income['high'] & credit_score['low'], risk['high'])
    rule7 = ctrl.Rule(income['low'] & debt_to_income['middle'] & credit_score['middle'], risk['middle'])
    rule8 = ctrl.Rule(income['high'] & debt_to_income['middle'] & credit_score['middle'], risk['low'])
    
    risk_ctrl = ctrl.ControlSystem([rule1, rule2, rule3, rule4, rule5, rule6, rule7, rule8])
    return ctrl.ControlSystemSimulation(risk_ctrl)

# Function to generate synthetic data with a default value if no rules are activated
def generate_synthetic_data(fuzzy_system, n_samples=100, default_risk_value=5.0):
    np.random.seed(42)  # For reproducibility
    samples = []
    for _ in range(n_samples):
        income_val = np.random.randint(0, 100001)
        dti_val = np.random.randint(0, 101)
        credit_score_val = np.random.randint(300, 851)
        
        # Set inputs to the fuzzy system
        fuzzy_system.input['income'] = income_val
        fuzzy_system.input['debt_to_income'] = dti_val
        fuzzy_system.input['credit_score'] = credit_score_val
        
        try:
            fuzzy_system.compute()
            # Use computed risk if available; otherwise, use default value
            risk_val = fuzzy_system.output.get('risk', default_risk_value)
            samples.append([income_val, dti_val, credit_score_val, risk_val])
        except KeyError:
            print(f"No rule activated for income={income_val}, debt_to_income={dti_val}, credit_score={credit_score_val}. Using default risk value.")
            samples.append([income_val, dti_val, credit_score_val, default_risk_value])
    
    return pd.DataFrame(samples, columns=['income', 'debt_to_income', 'credit_score', 'risk'])

# Generate synthetic data for both systems
fuzzy_system_1 = create_fuzzy_system_1()
fuzzy_system_2 = create_fuzzy_system_2()

data1 = generate_synthetic_data(fuzzy_system_1, n_samples=100, default_risk_value=5.0)
data2 = generate_synthetic_data(fuzzy_system_2, n_samples=100, default_risk_value=5.0)

# Train and evaluate models on data from each fuzzy system
def train_and_evaluate(data):
    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(
        data[['income', 'debt_to_income', 'credit_score']], data['risk'], test_size=0.3, random_state=42)

    # Train a Random Forest model
    rf = RandomForestRegressor(n_estimators=100, random_state=42)
    rf.fit(X_train, y_train)

    # Predict and evaluate
    y_pred = rf.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    return mse, r2

# Evaluate both systems
mse1, r2_1 = train_and_evaluate(data1)
mse2, r2_2 = train_and_evaluate(data2)

# Store results in a DataFrame
evaluation_results = pd.DataFrame({
    "Fuzzy System": ["System 1", "System 2"],
    "Mean Squared Error (MSE)": [mse1, mse2],
    "R-squared (R2)": [r2_1, r2_2]
})

# Display the evaluation results
print("\n### Evaluation Results ###")
print(evaluation_results)

# Display the first few synthetic samples generated by each system
print("\n### Synthetic Data from Fuzzy Logic System 1 ###")
print(data1.head())

print("\n### Synthetic Data from Fuzzy Logic System 2 ###")
print(data2.head())



### Evaluation Results ###
  Fuzzy System  Mean Squared Error (MSE)  R-squared (R2)
0     System 1                  1.203119        0.783519
1     System 2                  0.531786        0.876308

### Synthetic Data from Fuzzy Logic System 1 ###
   income  debt_to_income  credit_score      risk
0   15795              92           570  7.622807
1   76820              82           514  5.000000
2   37194              74           387  7.804477
3   44131              23           430  7.546576
4   67221              52           643  2.136193

### Synthetic Data from Fuzzy Logic System 2 ###
   income  debt_to_income  credit_score      risk
0   15795              92           570  7.622807
1   76820              82           514  7.630028
2   37194              74           387  8.120495
3   44131              23           430  7.546576
4   67221              52           643  3.113306


In [5]:
import pandas as pd
import numpy as np
import skfuzzy as fuzz
import skfuzzy.control as ctrl
from sklearn.metrics import mean_squared_error, r2_score

# Define fuzzy logic system 1
def create_fuzzy_system_1():
    income = ctrl.Antecedent(np.arange(0, 100001, 1000), 'income')
    debt_to_income = ctrl.Antecedent(np.arange(0, 101, 1), 'debt_to_income')
    credit_score = ctrl.Antecedent(np.arange(300, 851, 1), 'credit_score')
    risk = ctrl.Consequent(np.arange(0, 11, 1), 'risk')
    
    income['low'] = fuzz.trimf(income.universe, [0, 0, 50000])
    income['high'] = fuzz.trimf(income.universe, [30000, 100000, 100000])
    debt_to_income['low'] = fuzz.trimf(debt_to_income.universe, [0, 0, 40])
    debt_to_income['high'] = fuzz.trimf(debt_to_income.universe, [20, 100, 100])
    credit_score['low'] = fuzz.trimf(credit_score.universe, [300, 300, 600])
    credit_score['high'] = fuzz.trimf(credit_score.universe, [550, 850, 850])
    risk['low'] = fuzz.trimf(risk.universe, [0, 0, 5])
    risk['high'] = fuzz.trimf(risk.universe, [5, 10, 10])
    
    rule1 = ctrl.Rule(income['high'] & debt_to_income['low'] & credit_score['high'], risk['low'])
    rule2 = ctrl.Rule(income['low'] & debt_to_income['high'] & credit_score['low'], risk['high'])
    rule3 = ctrl.Rule(income['high'] & debt_to_income['high'] & credit_score['high'], risk['low'])
    rule4 = ctrl.Rule(income['low'] & debt_to_income['low'] & credit_score['high'], risk['low'])
    rule5 = ctrl.Rule(income['low'] & debt_to_income['high'] & credit_score['high'], risk['high'])
    
    risk_ctrl = ctrl.ControlSystem([rule1, rule2, rule3, rule4, rule5])
    return ctrl.ControlSystemSimulation(risk_ctrl)

# Define fuzzy logic system 2
def create_fuzzy_system_2():
    income = ctrl.Antecedent(np.arange(0, 100001, 1000), 'income')
    debt_to_income = ctrl.Antecedent(np.arange(0, 101, 1), 'debt_to_income')
    credit_score = ctrl.Antecedent(np.arange(300, 851, 1), 'credit_score')
    risk = ctrl.Consequent(np.arange(0, 11, 1), 'risk')
    
    income['low'] = fuzz.trimf(income.universe, [0, 0, 50000])
    income['middle'] = fuzz.trimf(income.universe, [20000, 50000, 80000])
    income['high'] = fuzz.trimf(income.universe, [30000, 100000, 100000])
    debt_to_income['low'] = fuzz.trimf(debt_to_income.universe, [0, 0, 40])
    debt_to_income['middle'] = fuzz.trimf(debt_to_income.universe, [20, 50, 80])
    debt_to_income['high'] = fuzz.trimf(debt_to_income.universe, [20, 100, 100])
    credit_score['low'] = fuzz.trimf(credit_score.universe, [300, 300, 600])
    credit_score['middle'] = fuzz.trimf(credit_score.universe, [500, 675, 800])
    credit_score['high'] = fuzz.trimf(credit_score.universe, [550, 850, 850])
    risk['low'] = fuzz.trimf(risk.universe, [0, 0, 5])
    risk['middle'] = fuzz.trimf(risk.universe, [3, 5, 7])
    risk['high'] = fuzz.trimf(risk.universe, [5, 10, 10])
    
    rule1 = ctrl.Rule(income['high'] & debt_to_income['low'] & credit_score['high'], risk['low'])
    rule2 = ctrl.Rule(income['low'] & debt_to_income['high'] & credit_score['low'], risk['high'])
    rule3 = ctrl.Rule(income['middle'] & debt_to_income['middle'] & credit_score['middle'], risk['middle'])
    rule4 = ctrl.Rule(income['high'] & debt_to_income['high'] & credit_score['high'], risk['low'])
    rule5 = ctrl.Rule(income['low'] & debt_to_income['low'] & credit_score['high'], risk['low'])
    rule6 = ctrl.Rule(income['middle'] & debt_to_income['high'] & credit_score['low'], risk['high'])
    rule7 = ctrl.Rule(income['low'] & debt_to_income['middle'] & credit_score['middle'], risk['middle'])
    rule8 = ctrl.Rule(income['high'] & debt_to_income['middle'] & credit_score['middle'], risk['low'])
    
    risk_ctrl = ctrl.ControlSystem([rule1, rule2, rule3, rule4, rule5, rule6, rule7, rule8])
    return ctrl.ControlSystemSimulation(risk_ctrl)

# Function to generate synthetic data and add true risk scores
def generate_synthetic_data(fuzzy_system, n_samples=100, default_risk_value=5.0):
    np.random.seed(42)  # For reproducibility
    samples = []
    for _ in range(n_samples):
        income_val = np.random.randint(0, 100001)
        dti_val = np.random.randint(0, 101)
        credit_score_val = np.random.randint(300, 851)
        
        # Generate a true risk score for this sample (e.g., based on some formula or function)
        true_risk = (income_val / 100000) * 4 + (dti_val / 100) * 3 + (credit_score_val / 850) * 3

        # Set inputs to the fuzzy system
        fuzzy_system.input['income'] = income_val
        fuzzy_system.input['debt_to_income'] = dti_val
        fuzzy_system.input['credit_score'] = credit_score_val
        
        try:
            fuzzy_system.compute()
            # Use computed risk if available; otherwise, use default value
            risk_val = fuzzy_system.output.get('risk', default_risk_value)
            samples.append([income_val, dti_val, credit_score_val, risk_val, true_risk])
        except KeyError:
            print(f"No rule activated for income={income_val}, debt_to_income={dti_val}, credit_score={credit_score_val}. Using default risk value.")
            samples.append([income_val, dti_val, credit_score_val, default_risk_value, true_risk])
    
    return pd.DataFrame(samples, columns=['income', 'debt_to_income', 'credit_score', 'fuzzy_risk', 'true_risk'])

# Generate synthetic data for both systems
fuzzy_system_1 = create_fuzzy_system_1()
fuzzy_system_2 = create_fuzzy_system_2()

# Create datasets with true risk scores for both systems
data1 = generate_synthetic_data(fuzzy_system_1, n_samples=100, default_risk_value=5.0)
data2 = generate_synthetic_data(fuzzy_system_2, n_samples=100, default_risk_value=5.0)

# Function to evaluate fuzzy system performance against true risk scores
def evaluate_fuzzy_system(data):
    mse = mean_squared_error(data['true_risk'], data['fuzzy_risk'])
    r2 = r2_score(data['true_risk'], data['fuzzy_risk'])
    return mse, r2

# Evaluate both systems
mse1, r2_1 = evaluate_fuzzy_system(data1)
mse2, r2_2 = evaluate_fuzzy_system(data2)

# Store results in a DataFrame
evaluation_results = pd.DataFrame({
    "Fuzzy System": ["System 1", "System 2"],
    "Mean Squared Error (MSE)": [mse1, mse2],
    "R-squared (R2)": [r2_1, r2_2]
})

# Display the evaluation results
print("\n### Evaluation Results Against True Risk Scores ###")
print(evaluation_results)

# Display the first few synthetic samples with true risk scores and fuzzy logic predictions for each system
print("\n### Synthetic Data with True and Fuzzy Risk Scores from Fuzzy Logic System 1 ###")
print(data1.head())

print("\n### Synthetic Data with True and Fuzzy Risk Scores from Fuzzy Logic System 2 ###")
print(data2.head())


### Evaluation Results Against True Risk Scores ###
  Fuzzy System  Mean Squared Error (MSE)  R-squared (R2)
0     System 1                 12.329571       -3.330637
1     System 2                 11.757921       -3.129851

### Synthetic Data with True and Fuzzy Risk Scores from Fuzzy Logic System 1 ###
   income  debt_to_income  credit_score  fuzzy_risk  true_risk
0   15795              92           570    7.622807   5.403565
1   76820              82           514    5.000000   7.346918
2   37194              74           387    7.804477   5.073642
3   44131              23           430    7.546576   3.972887
4   67221              52           643    2.136193   6.518252

### Synthetic Data with True and Fuzzy Risk Scores from Fuzzy Logic System 2 ###
   income  debt_to_income  credit_score  fuzzy_risk  true_risk
0   15795              92           570    7.622807   5.403565
1   76820              82           514    7.630028   7.346918
2   37194              74           387    8.